In [1]:
from langgraph.graph import StateGraph, START, END
from typing import TypedDict
from langchain_groq import ChatGroq
from langgraph.checkpoint.memory import InMemorySaver

from dotenv import load_dotenv


In [20]:
load_dotenv()
llm = ChatGroq(model="gemma2-9b-it")


In [21]:
class JokeState(TypedDict):
    topic: str
    joke: str
    explanation: str
    

In [22]:
def generate_joke(state: JokeState):
    prompt = f'generate a joke on the topic {state["topic"]}'
    response = llm.invoke(prompt).content
    return {'joke': response}

In [23]:
def generate_explanation(state: JokeState):
    prompt = f'write an explanation for the joke - {state["joke"]}'
    response= llm.invoke(prompt).content
    return {'explanation': response}

In [24]:
graph = StateGraph(JokeState)

graph.add_node("generate_joke", generate_joke)
graph.add_node("generate_explanation", generate_explanation)

graph.add_edge(START, "generate_joke")
graph.add_edge("generate_joke", "generate_explanation")
graph.add_edge("generate_explanation", END)

checkpointer = InMemorySaver()
workflow = graph.compile(checkpointer=checkpointer)



In [25]:
config = {"configurable": {"thread_id": "1"}}

workflow.invoke({'topic':'pizza'}, config=config)

{'topic': 'pizza',
 'joke': 'Why did the pizza slice go to the doctor? \n\nBecause it was feeling cheesy! 🍕🤕  \n',
 'explanation': 'This joke plays on the double meaning of the word "cheesy." \n\n* **Literal meaning:**  Pizza is made with cheese, so a pizza slice literally is cheesy.\n\n* **Figurative meaning:** "Cheesy" can also mean something corny, unoriginal, or embarrassing.\n\nThe joke sets up the expectation that the pizza slice is sick in a literal way.  However, the punchline reveals that it\'s feeling "cheesy" in the figurative sense, making it a silly and unexpected twist.  \n\n'}

In [26]:
workflow.get_state(config)

StateSnapshot(values={'topic': 'pizza', 'joke': 'Why did the pizza slice go to the doctor? \n\nBecause it was feeling cheesy! 🍕🤕  \n', 'explanation': 'This joke plays on the double meaning of the word "cheesy." \n\n* **Literal meaning:**  Pizza is made with cheese, so a pizza slice literally is cheesy.\n\n* **Figurative meaning:** "Cheesy" can also mean something corny, unoriginal, or embarrassing.\n\nThe joke sets up the expectation that the pizza slice is sick in a literal way.  However, the punchline reveals that it\'s feeling "cheesy" in the figurative sense, making it a silly and unexpected twist.  \n\n'}, next=(), config={'configurable': {'thread_id': '1', 'checkpoint_ns': '', 'checkpoint_id': '1f06dde1-5203-6ee2-8002-22cd307fd7d5'}}, metadata={'source': 'loop', 'step': 2, 'parents': {}}, created_at='2025-07-31T07:15:05.308745+00:00', parent_config={'configurable': {'thread_id': '1', 'checkpoint_ns': '', 'checkpoint_id': '1f06dde1-4ca2-6350-8001-436c5dfe8c99'}}, tasks=(), interru

In [27]:
list(workflow.get_state_history(config))

[StateSnapshot(values={'topic': 'pizza', 'joke': 'Why did the pizza slice go to the doctor? \n\nBecause it was feeling cheesy! 🍕🤕  \n', 'explanation': 'This joke plays on the double meaning of the word "cheesy." \n\n* **Literal meaning:**  Pizza is made with cheese, so a pizza slice literally is cheesy.\n\n* **Figurative meaning:** "Cheesy" can also mean something corny, unoriginal, or embarrassing.\n\nThe joke sets up the expectation that the pizza slice is sick in a literal way.  However, the punchline reveals that it\'s feeling "cheesy" in the figurative sense, making it a silly and unexpected twist.  \n\n'}, next=(), config={'configurable': {'thread_id': '1', 'checkpoint_ns': '', 'checkpoint_id': '1f06dde1-5203-6ee2-8002-22cd307fd7d5'}}, metadata={'source': 'loop', 'step': 2, 'parents': {}}, created_at='2025-07-31T07:15:05.308745+00:00', parent_config={'configurable': {'thread_id': '1', 'checkpoint_ns': '', 'checkpoint_id': '1f06dde1-4ca2-6350-8001-436c5dfe8c99'}}, tasks=(), interr

In [ ]:
config2 = {"configurable": {"thread_id": "2"}}

workflow.invoke({'topic':'donald trump'}, config=config2)

{'topic': 'donal trump',
 'joke': "Why did Donald Trump bring a ladder to the debate? \n\nBecause he heard the other candidates were going to be *raising* the bar! \n\n\nLet me know if you'd like to hear another one! 😄 \n",
 'explanation': 'This joke plays on a few things:\n\n* **Donald Trump\'s height:** Trump is known for being shorter than many of his political opponents. The joke implies that he might need a ladder to reach the "bar" set by the other candidates.\n* **"Raising the bar":** This idiom means to set a higher standard or expectation. The joke humorously twists this meaning to suggest that Trump is literally bringing a ladder because he expects the other candidates to be physically raising the bar during the debate.\n* **Political satire:** The joke also subtly pokes fun at the nature of political debates, where candidates often try to outdo each other with increasingly ambitious promises and criticisms.\n\n\nLet me know if you\'d like to hear another joke! 😄 \n\n'}

In [32]:
workflow.get_state_history(config2)

<generator object Pregel.get_state_history at 0x0000026FB3C331C0>

In [33]:
workflow.get_state_history(config)

<generator object Pregel.get_state_history at 0x0000026FB3C33320>